## Training/ Testing data

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from time import time
from operator import itemgetter
import time
from scipy import stats
from sklearn.metrics import confusion_matrix
import itertools
import brew
from brew.base import Ensemble
from brew.combination.combiner import Combiner
from brew.stacking.stacker import EnsembleStack
from brew.stacking.stacker import EnsembleStackClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier

from sklearn import (preprocessing, metrics, cross_validation)

from sklearn.externals import joblib

from numba import autojit


C:\Users\hulabapp\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# upsample minority class

def upsample(data, labels):
    """
    This function is to upsample the data for the under-repressenting class with replacement. 
    The result will be a matrix with the same amount of species for each class.
    """
    
    from collections import defaultdict
    import random

    label_indices = defaultdict(lambda: list())
    for idx, label in enumerate(labels):
        label_indices[label].append(idx)

    largest_class_size = max(map(lambda l: len(l), label_indices.values()))

    upsampled_indices = []
    for label, indices in label_indices.items():
        sampled_indices = indices[:]
        while len(sampled_indices) < largest_class_size:
            sampled_indices.append(random.choice(indices))
        upsampled_indices.extend(sampled_indices)

    upsampled_labels = labels[upsampled_indices]
    upsampled_data = data[upsampled_indices, :]

    return upsampled_data, upsampled_labels



# Create training and testing set

In [23]:
# divide into test set and training set
def create_training_sample(datapath, normalize, count, server, up):
    """
    This function create training and testing set from our large training sample
    Output of the function will be X, Y, X_test, Y_test corresponding to training set and test set
    Training set will be upsampled to increase minority class occurance. Test set will not be.
    
    If normalize = true, normalize the data
    """
    
    start = time.time()
#     datapath = 'H:/MatlabProjects/Nate/Dropbox/Rotation 3 Project/Data/FullData/featurematrixandtarget_02h'
    datacontent = sio.loadmat(datapath)

    X_tot = datacontent['featm']
    X_tot = np.nan_to_num(X_tot) # replace nan with 0
    nsubjects = X_tot.shape[0]
    
    if normalize == True:
        X_tot = stats.zscore(X_tot)
        scaler = preprocessing.StandardScaler().fit(X_tot)
        X_tot = scaler.transform(X_tot)

    Yv = datacontent['y']
    Y_tot = np.ravel(Yv)

# datapath = 'D:/Dropbox (Nate)/Rotation 3 Project/Data/FullData_Agg/feature_05h.csv'
# datapath_y = 'D:/Dropbox (Nate)/Rotation 3 Project/Data/FullData_Agg/target_05h.csv'
# datapath= '/Users/ignacioperez-pozuelo/Dropbox/Rotation 3 Project/Data/FeatData/featurematrixandtarget_20h'
# X_tot = pd.read_csv(datapath)
# X_tot = np.nan_to_num(X_tot) # replace nan with 0

# Y_tot = pd.read_csv(datapath_y)
# Y_tot = np.ravel(Y_tot)

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=None)
    for train_index, test_index in sss.split(X_tot, Y_tot):
        X, X_test = X_tot[train_index], X_tot[test_index]
        Y, Y_test = Y_tot[train_index], Y_tot[test_index]
   

# Do we want to upsample the minority class or not?
    if up == True
        X,Y = upsample(X,Y)
    
    # make new folder for every hour of train-test data
    os.makedirs('%s/Dropbox/Rotation 3 Project/TrainTest/%02d' %(server,count))
#     X_test, Y_test = upsample(X_test, Y_test)
    np.savetxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/trainX.csv' %(server,count), X, delimiter=',')
    np.savetxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/trainY.csv' %(server,count), Y, delimiter=',')
    np.savetxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/testX.csv' %(server,count), X_test, delimiter=',')
    np.savetxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/testY.csv' %(server,count), Y_test, delimiter=',')
    return X,Y,X_test,Y_test


In [24]:
for count in range(1,25):
    datapath = '%s/Dropbox/Rotation 3 Project/Data/Training Data/featurematrixandtarget_%02dh'%(server,count)
    create_training_sample(datapath, False, count,server, True)
    